### ライブラリのimport

In [370]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import pandas as pd
from pandas_profiling import ProfileReport
%matplotlib inline
from preprocessing.initial_preprocessing import initial_preprocessing
from preprocessing.drop_emergency_suspicious_height_weight import drop_emergency_suspicious_height_weight
from preprocessing.calculate_bmi_bmr_category import calculate_bmi_bmr_category
from preprocessing.remove_duplicate_ctdi import remove_duplicate_ctdi
from preprocessing.rename_and_drop_order import rename_and_drop_order_one_scan_ctdi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 解析用の元データを読み込み

In [371]:
df = pd.read_excel('train_data.xlsx')

## データを前処理
* カラム名の変更
* 複数回スキャンについては,Mean CTDIvolが最大のみ残す
* 救急科などの怪しい身長と体重のデータを削除する

In [372]:
initial_preprocessing(df)
df = remove_duplicate_ctdi(df)
df.reset_index(drop=True, inplace=True)
# df_allで後で、身長体重を代入する分を分けられるようにしておく
df_all = df
df = drop_emergency_suspicious_height_weight(df)

正常に処理が行われました。


### 身長・体重の最終確認: この部分は自分で確認しないと危険

In [373]:
index = df['height_cm'].sort_values(ascending=True)[:30].index.to_list()

In [374]:
df[['scanning length', 'height_cm', 'weight_kg', 'id', 'study_date', 'target region', 'Mean CTDIvol']].loc[index]

,scanning length,height_cm,weight_kg,id,study_date,target region,Mean CTDIvol
2220,852.06,101.0,39.8,75206,2021-04-21,Chest,6.45
994,796.77,101.0,39.8,75206,2021-05-18,Chest,7.38
96,626.53,101.0,39.8,75206,2021-01-14,Abdomen,12.78
607,827.20,101.0,39.8,75206,2021-03-29,Chest,6.13
1179,796.77,101.0,39.8,75206,2021-06-18,Chest,6.70
2341,621.51,101.0,39.8,75206,2021-06-02,Abdomen,17.10
260,567.16,101.0,39.8,75206,2021-02-04,Chest,6.03
6360,506.63,117.7,76.4,9326611,2023-02-10,Bronchus,10.38
105,456.72,117.7,76.4,9326611,2021-01-15,Chest,9.14
3494,531.81,117.7,76.4,9326611,2022-01-21,Bronchus,9.90


In [375]:
# 最終的な怪しいデータに関しては直接データを確認して削除する
drop_ids = [75206, 9326611, 17049]
for ids in drop_ids:
    df = df[~(df['id'] == ids)]
df.reset_index(drop=True, inplace=True)

### 身長体重予測、線量予測、異常検知でも使用しない項目を削除する

In [376]:
unuse_cols = ['hospital_ward', 'modality', 'adult_child', 'department', 'preset_name']
df.drop(labels=unuse_cols, axis=1, inplace=True)

In [362]:
df.to_excel('preprocess_all_scan_data.xlsx')

### 新たな特徴量を作成する

* BMI(Body Mass Index)  
  $BMI = weight(kg) / height(m)^2$  
  
* BMR(Basal Metabolic Rate: 基礎代謝率): 年齢、性別、身長、体重に基づいて計算される。ハリス・ベネディクト方程式と呼ばれる  
  男性$BMR=66.47 + (13.75*weight[kg]) + (5.003*height[cm]) - (6.755*age[歳])$  
  女性$BMR=655.1 + (9.563*weight[kg]) + (1.85*height[cm]) - (4.676*age[歳])$  
  
* 体格指数(Rohrer's index) ：学童期に適するとあるので、今回は使わない  
  $Rohrer's index = weight[kg] / height[m]^3$

体格指数（BMI）: 体重（kg）を身長（m）の二乗で割ったもので、以下のように分類されます（成人向けのWHOのガイドライン）。

18.5未満: 低体重（痩せ型）: underweight    
18.5～24.9: 標準体重: normal weight   
25.0～29.9: 肥満（1度）:obesity class1  
30.0～34.9: 肥満（2度）:obesity class2  
35.0～39.9: 肥満（3度）:obesity class3    
40以上: 肥満（4度、重症）:obesity class4

In [56]:
from preprocessing.calculate_bmi_bmr_category import calculate_bmi_bmr_category

In [57]:
calculate_bmi_bmr_category(df)

In [377]:
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '頸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部〜骨盤CT'))]
df = df[~((df['CTDIw phantom type'] == 'IEC Head Dosimetry Phantom') & (df['scan_area'] == '胸部CT'))]

* データに含まれている外れ値のデータはさらに削除しても良いかも

In [378]:
# 外れ値として取り扱うデータを追加
drop_accession_list = [1249030520220420, 1351460220230315, 1346485720230228, 1348835920230307,
                       1229782420220221]
for accession in drop_accession_list:
    df = df[~(df['accession'] == accession)]
df.reset_index(drop=True, inplace=True)

In [379]:
# 不適切な心臓関係のデータを取り除く
# 1.　管電圧が100kV, 2.肺静脈(カルト)で管電圧が140kVのもの,その他排除すべきデータ
#differenceが10以上のデータを確認した中で、以下のようなデータを削除 
# *脳でHead restを使用せずベッドで撮影したデータ 
# *オーダーに対して不適切な部位が撮影範囲に含まれている（脳CTで肩が含まれている等） 
# *腕を挙上していない（体幹部撮影の際） *上肢撮影で体幹が含まれている *金属等が装着されている *オーダーと撮影部位が異なる
unuse_accession = [1348557720230306, 1106913420210113, 1112271520210127, 1116921520210210, 1130557720210324,
                   1163646020210707, 1175490820210818, 1216316120220114, 1150306520210521, 1273513820220714, 
                   1301969120221017, 1283505320220817, 1315170020221125, 1184192220210917, 1293945820220920,
                   1257936320220523, 1190750720211013, 1219548120220124, 1364809020230425, 1117860420210215,
                   1249639320220422, 1335762120230126, 1291697020220912, 1359568020230411, 1190377920211012,
                   1262435120220608, 1347736320230303, 1289845120220906, 1223184520220202, 1330730720230113,
                   1327520120230104, 1197756320211109, 1129424720210322, 1309065120221107, 1343513920230218,
                   1198504120211111, 1208625320211216, 1358077220230406, 1130613820210324, 1223603420220203,
                   1197519820211108, 1105253820210108, 1140475720210420, 1335584020230126, 1293219420220916,
                   1144301520210430, 1111324820210125, 1338397620230204, 1164256820210709, 1323580920221220]
for accession in unuse_accession:
    df = df[~(df['accession'] == accession)]
df.reset_index(drop=True, inplace=True)

### スキャンオーダーの名前を変更する

In [380]:
rename_and_drop_order_one_scan_ctdi(df)

### 前処理が終了したデータを保存する。

In [382]:
df.to_excel('preprocessed_train_data.xlsx', index=False)